<a href="https://colab.research.google.com/github/tara-shukla/jrw/blob/main/multi_mbart_model_jrw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install numba

#from numba import cuda
#device = cuda.get_current_device()
#device.reset()


#PYTORCH_CUDA_ALLOC_CONF=expandable_segments = True


In [ ]:
!pip install datasets
!pip install transformers[torch]
! pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from datasets import load_dataset
import json
import os
import glob



#get train data file
!wget -q -O train.txt "https://docs.google.com/uc?export=download&id=1cQBB074e3-xffKNuDgUok5M_u7iuykYZ"

#get train corrected data file
!wget -q -O test.txt 'https://docs.google.com/uc?export=download&id=1P0IJRSSwxgAMPJP98bHnJNgzHqG-PfRl'

#get val data file
!wget -q -O val.txt 'https://docs.google.com/uc?export=download&id=18AvCV0nwvxSUA8U102enCqzdsLHsf04E'

#splitting text data and putting into json format
def formatdata(filename):
    with open((filename+".txt"), "r", encoding = 'utf-8') as file:
        data = []
        for line in file:
            error_sentence, correct_sentence = line.strip().split('\t')
            data.append({'error_sentence': error_sentence, 'correct_sentence': correct_sentence})

        indexed_data = []
        for index, pair in enumerate(data):
            indexed_pair = {'index': index, **pair}
            indexed_data.append(indexed_pair)

    json_data = json.dumps(indexed_data, ensure_ascii=False, indent=4)

    # Write JSON data
    with open((filename+".json"), "w", encoding = 'utf-8') as json_file:
        json_file.write(json_data)


formatdata("train")
formatdata("test")
formatdata("val")

#loading dataset
raw_datasets = load_dataset("json", data_files={'train': 'train.json', 'test': 'test.json', 'val':'val.json'})


raw_datasets = raw_datasets.filter(lambda x: x["correct_sentence"] is not None)
raw_datasets = raw_datasets.filter(lambda x: x["error_sentence"] is not None)
data_sample = raw_datasets["train"].shuffle(seed=63).select(range(1000))
# Peek at data
data_sample[:3]

raw_train_dataset = raw_datasets["train"]
#need to index like in tutorial? labels??
raw_train_dataset[0]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/19898 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4265 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4264 [00:00<?, ? examples/s]

Filter:   0%|          | 0/19898 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4265 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4264 [00:00<?, ? examples/s]

{'error_sentence': '사람들이 정년을 연장은 고령화 사회의 해결 방법이라고 생각했다.',
 'index': 0,
 'correct_sentence': '사람들이 정년 연장은 고령화 사회의 해결 방법이라고 생각했다.'}

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang="ko_KR", tgt_lang="ko_KR")


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = examples["error_sentence"]
    targets = examples["correct_sentence"]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=128, truncation=True
    )
    return model_inputs


tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

tokenized_datasets

Map:   0%|          | 0/19898 [00:00<?, ? examples/s]

Map:   0%|          | 0/4265 [00:00<?, ? examples/s]

Map:   0%|          | 0/4264 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 19898
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4265
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4264
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()
batch["decoder_input_ids"]

tensor([[     2, 250014,  14238,    138,   2680, 147886,  25006,  12812,   8280,
           7084,      6, 151327,   7063,   1963,   5837,   2166,  74534,  12624,
          25006,  12812,    469,  53941,  82017,   3156,   6571,   2461,   1020,
          92234,  92462,      5],
        [     2, 250014, 188425,   3184,  90546,    713,      6,  20427,  97854,
          34682,  11817,   2461,   1020,   2901,  76516, 133299,  29044,     32,
              2,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1]])

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"outputdir_me",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    push_to_hub=False,
    report_to="wandb"
)

In [ ]:
from nltk.translate.gleu_score import sentence_gleu as gleu_score
from sklearn.metrics import precision_recall_fscore_support
!pip install evaluate
import evaluate
!pip install sacrebleu

gleu_metric = evaluate.load("google_bleu")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

bleu_metric = evaluate.load("sacrebleu")
metrics = [gleu_metric, precision_metric, recall_metric, f1_metric]


def compute_metrics(eval_preds):
    predictions, labels = eval_preds

    # In case the model returns more than the prediction logits
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    gleu = gleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    bleu = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)


    #for p, l in zip(decoded_preds, decoded_labels):
        #print(f"pred: {p}, label: {l}")

    precision, recall, f1, _ = precision_recall_fscore_support(decoded_labels, decoded_preds, average='weighted')
    #gleu = gleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    #precision = precision_metric.compute(predictions=decoded_preds, references=decoded_labels)
    #recall = recall_metric.compute(predictions=decoded_preds, references=decoded_labels)
    #f1=f1_metric.compute(predictions=decoded_preds, references=decoded_labels)



    return {
      "bleu": bleu["score"],
      "gleu": gleu["google_bleu"],
      "f1": f1,
      "precision": precision,
      "recall": recall
      }


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00


In [ ]:
#make trainer
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

trainer.evaluate(max_length=128)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


{'eval_loss': 5.297412872314453,
 'eval_bleu': 49.22257016310133,
 'eval_gleu': 0.5234302213940786,
 'eval_f1': 0.0017979987492182613,
 'eval_precision': 0.0017589118198874296,
 'eval_recall': 0.001876172607879925,
 'eval_runtime': 702.4651,
 'eval_samples_per_second': 6.07,
 'eval_steps_per_second': 0.759}

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Bleu,Gleu,F1,Precision,Recall
1,0.488000,0.420414,65.351633,0.675370,0.218340,0.218457,0.218340
2,0.288700,0.403445,66.820403,0.688137,0.250078,0.250235,0.250000
3,0.159800,0.446258,66.711298,0.686252,0.260632,0.260553,0.260788


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 

TrainOutput(global_step=7464, training_loss=0.3491484201478294, metrics={'train_runtime': 6512.3722, 'train_samples_per_second': 9.166, 'train_steps_per_second': 1.146, 'total_flos': 4242093030604800.0, 'train_loss': 0.3491484201478294, 'epoch': 3.0})

In [ ]:
trainer.evaluate(max_length=128)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.4462580680847168,
 'eval_bleu': 66.7112983861816,
 'eval_gleu': 0.6862517516171496,
 'eval_f1': 0.2606316447779862,
 'eval_precision': 0.26055347091932457,
 'eval_recall': 0.2607879924953096,
 'eval_runtime': 616.1504,
 'eval_samples_per_second': 6.92,
 'eval_steps_per_second': 0.865,
 'epoch': 3.0}

In [ ]:
#next(model.parameters()).is_cuda

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.46276962757110596,
 'eval_bleu': 66.26108341029281,
 'eval_gleu': 0.6823331260002655,
 'eval_f1': 0.24282923016803434,
 'eval_precision': 0.24279015240328253,
 'eval_recall': 0.24290738569753811,
 'eval_runtime': 614.9753,
 'eval_samples_per_second': 6.935,
 'eval_steps_per_second': 0.868,
 'epoch': 3.0}

In [ ]:
import torch

def finetuned_generate(text, model, tokenizer):
  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
  corrected_ids = model.generate(torch.tensor([input_ids]),
                                  max_length=128,
                                  eos_token_id=1, num_beams=4,
                                  early_stopping=True, repetition_penalty=2.0)

  #output_text = tokenizer.decode(corrected_ids.squeeze().tolist(), skip_special_tokens=True)
  #return output_text
  output_texts = []
  for ids in corrected_ids:
      decoded_text = tokenizer.decode(ids, skip_special_tokens=True)
      sentences = decoded_text.split('.')  # Split into sentences
      for sentence in sentences:
          if sentence.strip():  # Check if the sentence is not empty
              output_texts.append(sentence.strip()+".")



  return output_texts


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to('cpu')
text = '한국어이어렵다고 생각하고 잇다.'
output_text=finetuned_generate(text, model, tokenizer)
print(output_text)


['한국어가 어렵다고 생각하고 있다.', '한국어가 어렵다.', '한국어가 어렵습니다.', '한국어가 어렵다고 생각한다.', '한국어이어렵다고 생각하고 싶다.', '한국어가 어렵다.', '한국어가 어렵다.', '한국어가 어렵다고싶다.', '한국어가 어렵다.']


AttributeError: 'str' object has no attribute 'type'

In [ ]:


# Save the model to Google Drive
model.save_pretrained('/content/drive/My Drive/mbart_model')
tokenizer.save_pretrained('/content/drive/My Drive/mbart_model')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


('/content/drive/My Drive/mbart_model/tokenizer_config.json',
 '/content/drive/My Drive/mbart_model/special_tokens_map.json',
 '/content/drive/My Drive/mbart_model/sentencepiece.bpe.model',
 '/content/drive/My Drive/mbart_model/added_tokens.json',
 '/content/drive/My Drive/mbart_model/tokenizer.json')

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_path = "/content/drive/My Drive/mbart_model"
tokenizer_path = "/content/drive/My Drive/mbart_model"
config_path = "/content/drive/My Drive/mbart_model"


model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = MBart50TokenizerFast.from_pretrained(tokenizer_path, src_lang="ko_KR", tgt_lang="ko_KR")



In [ ]:
import torch
from nltk.translate.gleu_score import sentence_gleu as gleu_score
import nltk
nltk.download('punkt')

def calculate_gleu_score(candidate_text, reference_text):

  return gleu_score([reference_text], candidate_text)

def finetuned_generate(text, model, tokenizer, reference_text):
    raw_input_ids = tokenizer.encode(text)
    input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
    corrected_ids = model.generate(torch.tensor([input_ids]),
                                    max_length=128,
                                    eos_token_id=1, num_beams=4,
                                    early_stopping=True, repetition_penalty=2.0,
                                    num_return_sequences=4)  # Change the number of return sequences as needed


    output_texts = []
    for ids in corrected_ids:
        decoded_text = tokenizer.decode(ids, skip_special_tokens=True)
        sentences = decoded_text.split('.')  # Split into sentences
        for sentence in sentences:
            if sentence.strip():  # Check if the sentence is not empty
                output_texts.append(sentence.strip()+".")




    best_gleu_score = float('-inf')
    best_candidate = None
    for sentence in output_texts:

        gleu_score = calculate_gleu_score(sentence, reference_text)
        if gleu_score > best_gleu_score:
            best_gleu_score = gleu_score
            best_candidate = sentence

    return best_candidate

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
text = '한국어이어렵다고 생각하고 잇다.'
corrected = "한국어가 어렵다고 생각하고 있다."
output_text=finetuned_generate(text, model, tokenizer, corrected)
print(output_text)

한국어가 어렵다고 생각하고 있다.


In [ ]:
output_file_path = "mbart_corrected_dataset.txt"

with open(output_file_path, "w", encoding="utf-8") as output_file:
    # Iterate through each tuple in the test dataset
    for sentences in raw_datasets["test"]:
        correct_sentence = sentences["correct_sentence"]
        error_sentence = sentences["error_sentence"]
        index = sentences["index"]
        corrected_sentence = finetuned_generate(error_sentence, model, tokenizer, correct_sentence)

        # Write Corrected Sentence and Index to the Text File
        #output_file.write(f"Index: {index}\n")
        #output_file.write(f"Correct Sentence: {correct_sentence}\n")
        output_file.write(corrected_sentence)
        print(corrected_sentence)
        output_file.write("\n")

# Inform when all sentences are corrected and saved
print("All sentences corrected and saved to:", output_file_path)

본 이유로 TV에서 방영하는 영화의 방송 심의 심의와 찬성한다는 입장이다.
지금부터 방콕을 소개하겠다.
페이스북은 유투복을 많이 사용을 했었습니다.
예를 들면 웰빙 음식으로 치즈 빵을 들 수 있다.
미국에서 나는 매운 음식을 못 먹었다.
요즘 같아선 선생님들도 포기하고 싶을 것 같아요.
하지만 한국에서는 밥이 아주 비싸서 많이 밥을 못 사요.
도운 아프라카며 추운 시비리아니 마음에 들면 거기로 이사하는 것이 좋다.
나는 남자가 전업주부가 되는 것에 찬성한다.
그것은 인간의 우리가 지금까지 편하게 쉽게 생활할 수 있도록 자연을 소중하게 생각하지 않고 우리 손으로 그렇게 만든 것이다.
나무집을 짓을 때 나무로 오랫동안 살았던 것을 짓으면 좋겠습니다.
그럼 다음 달에 봐요! 그럼 다음 달에 보아요! 그럼 다음 달에 봐요! 그럼 다음 달에 또 봐요! 그럼 다음 달에 그럼 다음 달에 그럼 다음 달에 그럼 다음 달에 그럼 다음 달에 그럼 다음 달에 그럼 다음 달에 그럼 다음 달에 그럼 다음 달에 그럼 다음 달에.
그래서 직장에서 가야 한다고 생각한다.
시월 이십육일은 제 생일이에요.
저는 다음에 음식을 준비해야겠어요.
부모님께서는 고등학교에서 교사를 하시는데 항상 학생들을 생각하고 노력을 하시는 모습을 보며 존경하고 있다.
토요일 저녁에는 이태원에서 맥주를 마시면서 삼겹살을 먹을 거예요.
한국 영화가 아주 재미있습니다.
모두가 차를 타서 생활을 하다가는 공기가 나빠지거나 여름은 더 더 더 더.
그것은 창의적 광고로 상품 판매율을 높일 수 있냐고 질문했습니다.
일반인 중에도 마찬가지고 타인의 사생활을 보호하지 않고 공개하면 수많은 사건이 일어날 위험도 있다.
몸 관리도 할 수 있지만 마음이 먹어야 할 수 있다.
나는 러시아에서 자고 싶지 않아서 한국에서 살면 좋겠다.
커피뿐만 아니라 더울 때 아이스크림을 먹으러 가자고 말하는 사람도 많이 있었다 커피숍에 커피숍에 커피숍에 커피숍에 커피숍에 커피숍에 커피숍에 커피숍에 커피숍에 커피숍에 커피숍에 커피숍에 뿐만 아니라 더울 때 